# <center>NBA Analytics Data Analysis Project<center>

### <center>presented by Brian Zamkotowicz for Springboard Capstone Project<center>

In the following project I intend to explore the relationship between advanced NBA metrics and their value as a predictor of success.  Specifically, I will approach the business problem of whether or not some set of advanced metrics might be useful when trying to improve an NBA team on a year to year basis.  I will explore the relationship between these metrics and team wins both individually and in 3 dimensional space, and attempt to determine what metric or combination of metrics might be useful to an NBA general manager whose goal is to improve on the current year's win total in the coming year.


### <center>Part 1 -  Data Wrangling<center>
    
In order to to determine the relationship between NBA advanced metrics and and change in team wins (hereafter referred to as win delta), the first step is to gather the data.  Many statisticians have debated which statistics are the truest indicators of performance for years. As a result many of these metrics are readily available.  Basketball-Reference.com in particular is a great source of data. The following spreadsheet provided most of the player data that I wanted to explore:
https://drive.google.com/file/d/0Bx1NfCUslJwxNm00UnZXN2c0dTA/view?usp=sharing

In [1]:
# Import player statistacal data. Read into pandas dataframe, list available colums of data.

import pandas as pd
nba = pd.read_excel("C://NBA/BBRef.xlsm", Sheetname = None, skiprows = 9)
nba= nba[(nba['Year']> 2009) & (nba['Year']< 2016)]



In [2]:
list(nba)

['Year',
 'Tm',
 'Rk',
 'Player',
 'Age',
 'G',
 'MP',
 'PER',
 'TS%',
 '3PAr',
 'FTr',
 'ORB%',
 'DRB%',
 'TRB%',
 'AST%',
 'STL%',
 'BLK%',
 'TOV%',
 'USG%',
 '.',
 'OWS',
 'DWS',
 'WS',
 'WS/48',
 '..1',
 'OBPM',
 'DBPM',
 'BPM',
 'VORP',
 'OWS/48',
 'DWS/48',
 'Shot%',
 'Player ID',
 'Team Mar',
 'Team MP',
 'Team Gm',
 'Year 3PAr',
 'Team TS%',
 'Tm USG',
 'Tm TS W/O Plyr',
 'Reb/Vers',
 'Defense',
 'Val/Shot',
 'Offense',
 'MPG + Int',
 'RAW SPM',
 '%Min',
 'RAW Contrib',
 'Tm Sum',
 'Tm Adj',
 'BPM.1',
 'StdErr',
 'Contrib',
 'VORP.1',
 'Reb/Vers.1',
 'Val/Shot.1',
 'Offense.1',
 'Defense.1',
 'MPG',
 'Raw OBPM',
 'Contrib.1',
 'Tm Ortg',
 'Tm Sum.1',
 'Tm Adj.1',
 'Tm DRtg',
 'OBPM.1',
 'OStdErr',
 'Ocontrib',
 'OVORP',
 'DBPM.1',
 'DStdErr',
 'Dcontrib',
 'DVORP',
 'Sum SPM',
 '%Min.1',
 'MPG.1',
 'ReMPG',
 'BPM.2',
 'Contrib.2',
 'VORP.2',
 'VORP-Gm',
 'O-BPM',
 'Ocontrib.1',
 'OVORP.1',
 'OVORP-Gm',
 'D-BPM',
 'Dcontrib.1',
 'DVORP.1',
 'DVORP-Gm',
 'Production',
 'Prod-Gm',

##### <center>Name changes<center>

I am aware that several teams changed names during the 2011-2015 time frame that i chose to explore.  I counted values to determine which teams seemed to have multiple monikers in the time period.

In [3]:
nba['Tm'].value_counts()

WAS    122
BOS    118
CLE    118
HOU    118
NYK    114
PHI    112
MEM    110
SAC    110
SAS    110
MIL    110
DAL    109
MIN    109
LAC    108
TOR    108
OKC    107
GSW    107
MIA    105
POR    103
PHO    101
DEN    100
UTA    100
LAL     99
ATL     99
CHI     95
ORL     94
IND     94
DET     92
CHA     91
NOH     76
NJN     61
BRK     54
NOP     42
CHO     17
Name: Tm, dtype: int64

NOH, NJN, BRK, NOP, and CHO appear to have unusually low counts.  A bit of research indicates that the New Jersey Nets became the Brooklyn Nets and the New Orleans Hornets became the Pelicans.  Also the Charlotte Hornets are indicated by both the CHA and CHO abbreviations.

In [4]:
#Replace New Jersey Nets with new name of Brooklyn, and New Orleans Pelicans with Hornets, and merge 2 different Hornets abbreviations
nba['Tm'] = nba['Tm'].str.replace('NJN', 'BRK')
nba['Tm'] = nba['Tm'].str.replace('NOP', 'NOH')
nba['Tm'] = nba['Tm'].str.replace('CHO', 'CHA')
nba['Tm'].nunique()


30

Since there are 30 NBA teams it appears to be properly organized now

In [5]:
#create a new dataframe with only columns we want to look at (we can always edit this later) in order 
    #keep it readable
nba1= nba[['Year', 'Tm', 'Player', 'G', 'BPM', 'VORP', 'WORP', 'MP', 'PER','Yrs Experience', 'Age']].copy()

# adding a % of minutes played stat that we can use to normalize results later
nba1['MIN'] = (nba1['MP']/ (48* 82))

# change the Yrs experience column to an integer to make it easier to work with later
nba1['Yrs Experience'] = nba1['Yrs Experience'].astype(int)

nba1.reindex()
print(nba1.head())

         Year   Tm          Player     G  BPM  VORP       WORP      MP   PER  \
13646  2010.0  OKC    Kevin Durant  82.0  4.6   5.4  15.640804  3239.0  26.2   
13647  2010.0  PHI  Andre Iguodala  82.0  2.5   3.6  10.218036  3193.0  17.8   
13648  2010.0  MEM        Rudy Gay  80.0  0.3   1.8   5.197248  3175.0  16.2   
13649  2010.0  CHA  Gerald Wallace  76.0  3.1   4.0  11.658262  3119.0  18.3   
13650  2010.0  MEM       O.J. Mayo  82.0  0.4   1.9   5.089625  3113.0  14.6   

       Yrs Experience   Age       MIN  
13646               2  21.0  0.822917  
13647               5  26.0  0.811230  
13648               3  23.0  0.806657  
13649               8  27.0  0.792429  
13650               1  22.0  0.790904  


WORP, PER, and VORP are all stats that take the amount of time played into account.  Box Plus Minus is not (it essentially shows how well a player plays while he is in the game).  In order to get a useful number (since a player with great BPM that plays 2 minutes a game probably won't help a team improve much) I have chosen to turn BPM into BPM_A which is returns Box plus minus adjusted for minutes played.

Because I am exploring how previous stats affect the next year, I need variables that show the year being examined and also the year whose stats are being looked at.  So for example in 2011 I would be looking at statistics from 2010.  That previous year will be added as a variable called 'use year'  meaning 2010 statistics (for example) will be used in 2011.

In [6]:
#add 'BPM_A' to adjust box plus minus for minutes
nba1['BPM_A']= (nba1['BPM'] * nba1['MIN'])

#add a variable to that shows what year the data will be used for (i.e. 2011 data used to determine 2012 wins)
nba1['Use_Year'] = (nba['Year'] + 1).astype(int)

# convert Year to interger for later join
nba1['Year'] = nba1['Year'].astype(int)

# WORP and VORP are already intended to take minutes into account and should be 'plug and play' statistics
nba1.head()

,Year,Tm,Player,G,BPM,VORP,WORP,MP,PER,Yrs Experience,Age,MIN,BPM_A,Use_Year
13646,2010,OKC,Kevin Durant,82.0,4.6,5.4,15.640804,3239.0,26.2,2,21.0,0.822917,3.785417,2011
13647,2010,PHI,Andre Iguodala,82.0,2.5,3.6,10.218036,3193.0,17.8,5,26.0,0.811230,2.028074,2011
13648,2010,MEM,Rudy Gay,80.0,0.3,1.8,5.197248,3175.0,16.2,3,23.0,0.806657,0.241997,2011
13649,2010,CHA,Gerald Wallace,76.0,3.1,4.0,11.658262,3119.0,18.3,8,27.0,0.792429,2.456529,2011
13650,2010,MEM,O.J. Mayo,82.0,0.4,1.9,5.089625,3113.0,14.6,1,22.0,0.790904,0.316362,2011


In [7]:
# just grab an individual player to see what the data looks like

teams = pd.DataFrame(nba1)
teams = teams.sort_values(['Tm', 'Year'])
print(teams[teams['Player'].str.contains('Horford')])


       Year   Tm      Player     G  BPM  VORP       WORP      MP   PER  \
13667  2010  ATL  Al Horford  81.0  3.1   3.6  10.328078  2845.0  19.4   
14187  2011  ATL  Al Horford  77.0  3.8   4.0  11.762890  2704.0  20.7   
15071  2012  ATL  Al Horford  11.0  2.5   0.5   1.398342   348.0  19.0   
15239  2013  ATL  Al Horford  74.0  2.7   3.3   9.574574  2756.0  19.8   
15997  2014  ATL  Al Horford  29.0  2.3   1.0   3.101664   958.0  22.0   
16333  2015  ATL  Al Horford  76.0  3.8   3.4   9.252387  2318.0  21.4   

       Yrs Experience   Age       MIN     BPM_A  Use_Year  
13667               2  23.0  0.722815  2.240727      2011  
14187               3  24.0  0.686992  2.610569      2012  
15071               4  25.0  0.088415  0.221037      2013  
15239               5  26.0  0.700203  1.890549      2014  
15997               6  27.0  0.243394  0.559807      2015  
16333               7  28.0  0.588923  2.237907      2016  


In [8]:
# look at an example of a player whose rookie year falls within the timeframe we are exploring

teams = pd.DataFrame(nba1)
teams = teams.sort_values(['Tm', 'Year'])
print(teams[teams['Player'].str.contains('Cleanthony')])

       Year   Tm            Player     G  BPM  VORP      WORP     MP  PER  \
16630  2015  NYK  Cleanthony Early  39.0 -5.2  -0.5 -1.384375  647.0  8.4   

       Yrs Experience   Age      MIN     BPM_A  Use_Year  
16630               0  23.0  0.16438 -0.854776      2016  


In [9]:
# take a look at 1 year of stats for a player I know to be on multiple teams in a given year

teams = pd.DataFrame(nba1)
teams = teams.sort_values(['Tm', 'Year'])
print(teams[teams['Player'].str.contains('Tornike')])


       Year   Tm             Player     G  BPM  VORP      WORP     MP  PER  \
15677  2013  BRK  Tornike Shengelia  19.0 -5.5  -0.1 -0.278144   93.0  8.5   
16193  2014  BRK  Tornike Shengelia  17.0 -6.3  -0.1 -0.473875  137.0  3.3   
16255  2014  CHI  Tornike Shengelia   9.0 -4.4   0.0 -0.027618   17.0  9.0   

       Yrs Experience   Age       MIN     BPM_A  Use_Year  
15677               0  21.0  0.023628 -0.129954      2014  
16193               1  22.0  0.034807 -0.219284      2015  
16255               1  22.0  0.004319 -0.019004      2015  


In [10]:
# Create a data frame of just year, team and player
Player_list = teams[['Year', 'Tm', 'Player', 'Age']].copy()
Player_list.head()

,Year,Tm,Player,Age
13663,2010,ATL,Joe Johnson,28.0
13666,2010,ATL,Josh Smith,24.0
13667,2010,ATL,Al Horford,23.0
13707,2010,ATL,Marvin Williams,23.0
13709,2010,ATL,Jamal Crawford,29.0


In [11]:
#create a dataframe of relevant stats from previous year

prev_stats = teams[['Player', 'Use_Year', 'BPM', 'VORP', 'WORP', 'BPM_A', 'PER','G']]
prev_stats.head()

,Player,Use_Year,BPM,VORP,WORP,BPM_A,PER,G
13663,Joe Johnson,2011,2.2,3.0,8.177536,1.613110,19.3,76.0
13666,Josh Smith,2011,5.2,5.2,14.737892,3.792988,21.0,81.0
13667,Al Horford,2011,3.1,3.6,10.328078,2.240727,19.4,81.0
13707,Marvin Williams,2011,-0.1,1.2,3.075349,-0.062703,13.0,81.0
13709,Jamal Crawford,2011,0.9,1.8,4.733122,0.562500,18.4,79.0


In [12]:
# merge the data frame of year, player and team, with that players stats from the previous year

bball_data= Player_list.merge(prev_stats, left_on= ['Player', 'Year'], right_on= ['Player', 'Use_Year'])
bball_data.head()

,Year,Tm,Player,Age,Use_Year,BPM,VORP,WORP,BPM_A,PER,G
0,2011,ATL,Al Horford,24.0,2011,3.1,3.6,10.328078,2.240727,19.4,81.0
1,2011,ATL,Josh Smith,25.0,2011,5.2,5.2,14.737892,3.792988,21.0,81.0
2,2011,ATL,Joe Johnson,29.0,2011,2.2,3.0,8.177536,1.613110,19.3,76.0
3,2011,ATL,Jamal Crawford,30.0,2011,0.9,1.8,4.733122,0.562500,18.4,79.0
4,2011,ATL,Marvin Williams,24.0,2011,-0.1,1.2,3.075349,-0.062703,13.0,81.0


I was concerned about how certain situations could effect my data set.  These situations included players that played for more than one team in a season, and rookies (who have no previous year stats).

In [13]:
#this looks at the Knicks 2015 roster

knicks_2015 = bball_data[bball_data['Tm'] == 'NYK'] 
knicks_2015 = knicks_2015[knicks_2015['Year'] == 2015]
print(knicks_2015)

      Year   Tm             Player   Age  Use_Year  BPM  VORP       WORP  \
616   2015  NYK         J.R. Smith  29.0      2015  0.2   1.3   3.495611   
622   2015  NYK      Iman Shumpert  24.0      2015  0.1   1.0   2.549222   
716   2015  NYK  Amar'e Stoudemire  32.0      2015 -3.1  -0.4  -1.077327   
721   2015  NYK         Ricky Ledo  22.0      2015 -6.6   0.0  -0.117945   
1082  2015  NYK     Pablo Prigioni  37.0      2015  0.5   0.8   2.260442   
1087  2015  NYK       Alexey Shved  26.0      2015 -2.6  -0.1  -0.514731   
1797  2015  NYK       Shane Larkin  22.0      2015 -4.1  -0.3  -0.891464   
1798  2015  NYK        Jason Smith  28.0      2015 -3.4  -0.3  -0.867630   
1799  2015  NYK       Tim Hardaway  22.0      2015 -2.1  -0.1  -0.337202   
1800  2015  NYK    Carmelo Anthony  30.0      2015  3.3   4.0  11.440752   
1801  2015  NYK         Quincy Acy  24.0      2015 -1.5   0.1   0.246982   
1802  2015  NYK         Quincy Acy  24.0      2015  3.7   0.1   0.253450   
1803  2015  

In [14]:
#Quincy Acy appears twice.  I suspect this is because he was traded mid season

print(teams[teams['Player'].str.contains('Acy')])

       Year   Tm      Player     G  BPM  VORP      WORP      MP   PER  \
16497  2015  NYK  Quincy Acy  68.0 -3.1  -0.3 -0.925057  1287.0  11.9   
16029  2014  SAC  Quincy Acy  56.0 -1.5   0.1  0.246982   786.0   9.6   
15598  2013  TOR  Quincy Acy  29.0  0.5   0.2  0.629554   342.0  15.9   
16224  2014  TOR  Quincy Acy   7.0  3.7   0.1  0.253450    61.0  17.2   

       Yrs Experience   Age       MIN     BPM_A  Use_Year  
16497               2  24.0  0.326982 -1.013643      2016  
16029               1  23.0  0.199695 -0.299543      2015  
15598               0  22.0  0.086890  0.043445      2014  
16224               1  23.0  0.015498  0.057342      2015  


As expected Quincy Acy played for 2 different teams in 2014.  Since the stats I will be looking at (WORP, VORP, BPM_A) should be cumulative this using both rows of data for the 2015 Knicks is an appropriate solution.

While playing for multiple teams during a season seems to be a non-issue, the Knicks played 3 rookies on the 2015 team none of which appears in the teams stats. I'll examine the record for Cleanthony Early who I know played for the Knicks in 2015 I was able to query his record from the 'teams' dataframe earlier, but am unclear how or if it appears in the newer bball_data dataframe.

In [15]:
print(bball_data[bball_data['Player'].str.contains('Cleanthony')])

Empty DataFrame
Columns: [Year, Tm, Player, Age, Use_Year, BPM, VORP, WORP, BPM_A, PER, G]
Index: []


It appears rookie records were lost in the merge.  There was no "use year" 2015 for Cleanthony Early so it was lost in the join.

In [16]:
#attempt to make a new dataframe of just rookies

rookies = nba1[nba1['Yrs Experience'] == 0]
rookies.head()

,Year,Tm,Player,G,BPM,VORP,WORP,MP,PER,Yrs Experience,Age,MIN,BPM_A,Use_Year
13662,2010,GSW,Stephen Curry,80.0,1.0,2.2,5.979491,2896.0,16.3,0,21.0,0.735772,0.735772,2011
13680,2010,SAC,Tyreke Evans,72.0,1.5,2.3,6.617493,2677.0,18.2,0,20.0,0.680132,1.020198,2011
13682,2010,MIL,Brandon Jennings,82.0,1.1,2.1,5.140865,2671.0,14.5,0,20.0,0.678608,0.746468,2011
13723,2010,MIN,Jonny Flynn,81.0,-3.4,-0.8,-2.619733,2339.0,13.0,0,20.0,0.594258,-2.020478,2011
13737,2010,DET,Jonas Jerebko,80.0,-0.3,1.0,2.689474,2232.0,13.9,0,22.0,0.567073,-0.170122,2011


In [17]:
#change use year to the actual year the stats were compiled

rookies['Use_Year'] = (rookies['Use_Year']-1)
rookies.head()

C:\Conda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Year,Tm,Player,G,BPM,VORP,WORP,MP,PER,Yrs Experience,Age,MIN,BPM_A,Use_Year
13662,2010,GSW,Stephen Curry,80.0,1.0,2.2,5.979491,2896.0,16.3,0,21.0,0.735772,0.735772,2010
13680,2010,SAC,Tyreke Evans,72.0,1.5,2.3,6.617493,2677.0,18.2,0,20.0,0.680132,1.020198,2010
13682,2010,MIL,Brandon Jennings,82.0,1.1,2.1,5.140865,2671.0,14.5,0,20.0,0.678608,0.746468,2010
13723,2010,MIN,Jonny Flynn,81.0,-3.4,-0.8,-2.619733,2339.0,13.0,0,20.0,0.594258,-2.020478,2010
13737,2010,DET,Jonas Jerebko,80.0,-0.3,1.0,2.689474,2232.0,13.9,0,22.0,0.567073,-0.170122,2010


In [18]:
# concatenate the rookie data frame back to the nba_all data frame

nba_all = pd.concat([nba1, rookies])
nba_all.head()

,Year,Tm,Player,G,BPM,VORP,WORP,MP,PER,Yrs Experience,Age,MIN,BPM_A,Use_Year
13646,2010,OKC,Kevin Durant,82.0,4.6,5.4,15.640804,3239.0,26.2,2,21.0,0.822917,3.785417,2011
13647,2010,PHI,Andre Iguodala,82.0,2.5,3.6,10.218036,3193.0,17.8,5,26.0,0.811230,2.028074,2011
13648,2010,MEM,Rudy Gay,80.0,0.3,1.8,5.197248,3175.0,16.2,3,23.0,0.806657,0.241997,2011
13649,2010,CHA,Gerald Wallace,76.0,3.1,4.0,11.658262,3119.0,18.3,8,27.0,0.792429,2.456529,2011
13650,2010,MEM,O.J. Mayo,82.0,0.4,1.9,5.089625,3113.0,14.6,1,22.0,0.790904,0.316362,2011


In [19]:
# check to see if we now have 2 matching records for rookie year

print(nba_all[nba_all['Player'].str.contains('Cleanthony')])

       Year   Tm            Player     G  BPM  VORP      WORP     MP  PER  \
16630  2015  NYK  Cleanthony Early  39.0 -5.2  -0.5 -1.384375  647.0  8.4   
16630  2015  NYK  Cleanthony Early  39.0 -5.2  -0.5 -1.384375  647.0  8.4   

       Yrs Experience   Age      MIN     BPM_A  Use_Year  
16630               0  23.0  0.16438 -0.854776      2016  
16630               0  23.0  0.16438 -0.854776      2015  


In [20]:
#Rebuild a working data frame that include rookies

# rebuild prev_stats so that it includes 2 entries for each rookie year
prev_stats = nba_all[['Player', 'Use_Year', 'BPM', 'VORP', 'WORP', 'BPM_A', 'PER','G',]]

#merge it back to bball_data
bball_data= Player_list.merge(prev_stats, left_on= ['Player', 'Year'], right_on= ['Player', 'Use_Year'])
bball_data.head()

,Year,Tm,Player,Age,Use_Year,BPM,VORP,WORP,BPM_A,PER,G
0,2010,ATL,Jeff Teague,21.0,2010,-3.9,-0.3,-1.066509,-0.712424,11.0,71.0
1,2011,ATL,Al Horford,24.0,2011,3.1,3.6,10.328078,2.240727,19.4,81.0
2,2011,ATL,Josh Smith,25.0,2011,5.2,5.2,14.737892,3.792988,21.0,81.0
3,2011,ATL,Joe Johnson,29.0,2011,2.2,3.0,8.177536,1.613110,19.3,76.0
4,2011,ATL,Jamal Crawford,30.0,2011,0.9,1.8,4.733122,0.562500,18.4,79.0


In [21]:
#make sure Cleanthony Early now appears on the 2015 Knicks

knicks_2015 = bball_data[bball_data['Tm'] == 'NYK'] 
knicks_2015 = knicks_2015[knicks_2015['Year'] == 2015]
print(knicks_2015)

      Year   Tm             Player   Age  Use_Year  BPM  VORP       WORP  \
773   2015  NYK         J.R. Smith  29.0      2015  0.2   1.3   3.495611   
779   2015  NYK      Iman Shumpert  24.0      2015  0.1   1.0   2.549222   
886   2015  NYK  Amar'e Stoudemire  32.0      2015 -3.1  -0.4  -1.077327   
891   2015  NYK         Ricky Ledo  22.0      2015 -6.6   0.0  -0.117945   
1359  2015  NYK     Pablo Prigioni  37.0      2015  0.5   0.8   2.260442   
1370  2015  NYK       Alexey Shved  26.0      2015 -2.6  -0.1  -0.514731   
2221  2015  NYK       Shane Larkin  22.0      2015 -4.1  -0.3  -0.891464   
2222  2015  NYK        Jason Smith  28.0      2015 -3.4  -0.3  -0.867630   
2223  2015  NYK       Tim Hardaway  22.0      2015 -2.1  -0.1  -0.337202   
2224  2015  NYK  Langston Galloway  23.0      2015 -1.1   0.3   0.895954   
2225  2015  NYK    Carmelo Anthony  30.0      2015  3.3   4.0  11.440752   
2226  2015  NYK         Quincy Acy  24.0      2015 -1.5   0.1   0.246982   
2227  2015  

Cleanthony Early as well as other 2015 Knicks rookies Travis Wear and Langston Galloway now appear in the list of 2015 Knicks.

Now that the relevant statistics have been put together in the necessary rows, I can begin to combine individual player stats into teams.

In [22]:
#this adds up (sums) the relevant player metrics into team totals

nba_df =pd.DataFrame(bball_data.groupby(['Tm', 'Year'], as_index = False)['BPM_A', 'WORP', 'VORP', 'PER'].sum())
nba_df.head()

,Tm,Year,BPM_A,WORP,VORP,PER
0,ATL,2010,-0.712424,-1.066509,-0.3,11.0
1,ATL,2011,3.772002,43.820121,16.6,224.1
2,ATL,2012,-1.026931,26.987068,9.8,222.3
3,ATL,2013,-2.037678,28.595704,10.7,245.5
4,ATL,2014,0.746189,28.556464,10.4,265.2


The teams metrics have now been grouped together.  In order to explore their relationship with win delta, I need to now explored data about team wins.

In [23]:
#add list of nba teams including win totals for years 2011 to 2016 to a new dataframe

nba_teams = pd.read_excel("C://NBA/NBA_Standings_xlsx.xlsx")
nba_teams.head()

,Team,W,L,W/L%,GB,PS/G,PA/G,SRS,YR,ABBR
0,Cleveland Cavaliers* (1),57,25,0.695,—,104.3,98.3,5.45,2016,CLE
1,Toronto Raptors* (2),56,26,0.683,1,102.7,98.2,4.08,2016,TOR
2,Miami Heat* (3),48,34,0.585,9,100.0,98.4,1.50,2016,MIA
3,Atlanta Hawks* (4),48,34,0.585,9,102.8,99.2,3.49,2016,ATL
4,Boston Celtics* (5),48,34,0.585,9,105.7,102.5,2.84,2016,BOS


In [24]:
#check to make sure I have the same number of occurences for each team
nba_teams['ABBR'].value_counts()

PHI    6
SAS    6
TOR    6
OKC    6
DET    6
PHO    6
MIN    6
CHI    6
DEN    6
HOU    6
POR    6
CHA    6
SAC    6
ATL    6
UTA    6
BOS    6
GSW    6
LAC    6
CLE    6
ORL    6
MEM    6
LAL    6
MIL    6
WAS    6
MIA    6
DAL    6
NYK    6
IND    6
BRK    4
NOH    3
NOP    3
NJN    2
Name: ABBR, dtype: int64

###### Several teams don't show 6 years of data.  As I determined earlier,several teams changed names in the period of 2011 to 2016.  I once again replace the old names with the new to make them uniform.

In [25]:
#Replace New Jersey Nets with new name of Brooklyn, and New Orleans Pelicans with Hornets
nba_teams['ABBR'] = nba_teams['ABBR'].str.replace('NJN', 'BRK')
nba_teams['ABBR'] = nba_teams['ABBR'].str.replace('NOP', 'NOH')
nba_teams['ABBR'].value_counts()

PHI    6
UTA    6
TOR    6
OKC    6
DET    6
PHO    6
BRK    6
MIN    6
CHI    6
DEN    6
HOU    6
POR    6
CHA    6
SAC    6
ATL    6
SAS    6
NOH    6
GSW    6
LAC    6
CLE    6
ORL    6
MEM    6
LAL    6
MIL    6
WAS    6
MIA    6
DAL    6
NYK    6
IND    6
BOS    6
Name: ABBR, dtype: int64

In [26]:
# the following function can be used to compare a stat from one year with the same stat from the previous year

import numpy as np

def get_stat_diff(stat, df, sort1, sort2, row):
    stat_diff = np.nan
    prev_year_stat = list(df[(df[sort1]==row[sort1]) & (df[sort2] == row[sort2]-1)][stat].values)
    if prev_year_stat:
        stat_diff = row[stat] - prev_year_stat[0]
    return stat_diff

# determine change in WORP, VORP, BPM_A, and PER year on year and add to data frame nba_df

nba_df['WORP_Chg'] = nba_df.apply(lambda row: get_stat_diff('WORP', nba_df, 'Tm', 'Year', row), axis=1)
nba_df['VORP_Chg'] = nba_df.apply(lambda row: get_stat_diff('VORP', nba_df, 'Tm', 'Year', row), axis=1)
nba_df['BPM_A_Chg'] = nba_df.apply(lambda row: get_stat_diff('BPM_A', nba_df, 'Tm', 'Year', row), axis=1)
nba_df['PER_Chg'] = nba_df.apply(lambda row: get_stat_diff('PER', nba_df, 'Tm', 'Year', row), axis=1)

In [27]:
nba_df.head()

,Tm,Year,BPM_A,WORP,VORP,PER,WORP_Chg,VORP_Chg,BPM_A_Chg,PER_Chg
0,ATL,2010,-0.712424,-1.066509,-0.3,11.0,NaN,NaN,NaN,NaN
1,ATL,2011,3.772002,43.820121,16.6,224.1,44.886630,16.9,4.484426,213.1
2,ATL,2012,-1.026931,26.987068,9.8,222.3,-16.833053,-6.8,-4.798933,-1.8
3,ATL,2013,-2.037678,28.595704,10.7,245.5,1.608636,0.9,-1.010747,23.2
4,ATL,2014,0.746189,28.556464,10.4,265.2,-0.039241,-0.3,2.783867,19.7


In [28]:
#make a new dataframe that includes only the columns I need from team_wins:  Team, Year, and Wins

team_wins = nba_teams[['ABBR', 'YR', 'W']]
team_wins.tail()

,ABBR,YR,W
175,SAS,2011,61
176,DAL,2011,57
177,NOH,2011,46
178,MEM,2011,46
179,HOU,2011,43


In [29]:
team_wins.apply(lambda row: team_wins[(team_wins['ABBR']==row['ABBR']) & (team_wins['YR'] == row['YR']-1)]['W'].values, axis=1).tail()

175    []
176    []
177    []
178    []
179    []
dtype: object

In [30]:
# I can also use my get_stat_diff function to get a change in wins number from the team wins data frame

team_wins['Win_Delta'] = team_wins.apply(lambda row: get_stat_diff('W', team_wins, 'ABBR', 'YR', row), axis=1)

C:\Conda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [31]:
team_wins.head()

,ABBR,YR,W,Win_Delta
0,CLE,2016,57,4.0
1,TOR,2016,56,7.0
2,MIA,2016,48,11.0
3,ATL,2016,48,-12.0
4,BOS,2016,48,8.0


In [32]:
#merge the 2 dataframes based on team and year
hoops_data = nba_df.merge(team_wins, left_on = ['Tm', 'Year'], right_on = ['ABBR', 'YR'])


In [33]:
hoops_data.head()

,Tm,Year,BPM_A,WORP,VORP,PER,WORP_Chg,VORP_Chg,BPM_A_Chg,PER_Chg,ABBR,YR,W,Win_Delta
0,ATL,2011,3.772002,43.820121,16.6,224.1,44.886630,16.9,4.484426,213.1,ATL,2011,44,NaN
1,ATL,2012,-1.026931,26.987068,9.8,222.3,-16.833053,-6.8,-4.798933,-1.8,ATL,2012,40,-4.0
2,ATL,2013,-2.037678,28.595704,10.7,245.5,1.608636,0.9,-1.010747,23.2,ATL,2013,44,4.0
3,ATL,2014,0.746189,28.556464,10.4,265.2,-0.039241,-0.3,2.783867,19.7,ATL,2014,38,-6.0
4,ATL,2015,-1.173145,24.713699,9.1,218.3,-3.842764,-1.3,-1.919334,-46.9,ATL,2015,60,22.0


In [34]:
#make sure only 2011 records have NaN values

hoops_data[hoops_data.isnull().any(axis=1)]

,Tm,Year,BPM_A,WORP,VORP,PER,WORP_Chg,VORP_Chg,BPM_A_Chg,PER_Chg,ABBR,YR,W,Win_Delta
0,ATL,2011,3.772002,43.820121,16.6,224.1,44.886630,16.9,4.484426,213.1,ATL,2011,44,NaN
5,BOS,2011,3.521977,52.300859,19.4,311.4,53.099594,19.6,3.907190,290.3,BOS,2011,56,NaN
10,BRK,2011,-3.645655,22.654689,8.2,312.1,24.779622,8.9,-2.032241,300.6,BRK,2011,24,NaN
15,CHA,2011,-0.517480,23.636017,9.0,302.7,24.385604,9.2,0.187424,280.4,CHA,2011,34,NaN
20,CHI,2011,1.627388,38.244476,13.9,188.2,36.113133,13.1,2.272790,164.0,CHI,2011,62,NaN
25,CLE,2011,-4.736484,19.159213,7.7,269.4,19.341184,7.8,-4.607444,244.7,CLE,2011,19,NaN
30,DAL,2011,-0.510950,34.847730,12.8,265.2,33.267740,12.2,-0.742149,246.7,DAL,2011,57,NaN
35,DEN,2011,4.054700,52.344928,19.2,268.4,50.517718,18.5,4.054700,252.0,DEN,2011,50,NaN
40,DET,2011,-4.224898,15.260572,5.6,195.0,14.095675,5.2,-2.837932,160.8,DET,2011,30,NaN
45,GSW,2011,-2.075838,22.211455,8.4,263.9,15.857232,6.1,-2.135315,219.3,GSW,2011,36,NaN


In [35]:
hoops_data[hoops_data.isnull().any(axis=1)].count()


Tm           30
Year         30
BPM_A        30
WORP         30
VORP         30
PER          30
WORP_Chg     26
VORP_Chg     26
BPM_A_Chg    26
PER_Chg      26
ABBR         30
YR           30
W            30
Win_Delta     0
dtype: int64

The data now appears to be cleaned and organized in a fashion where I can start to explore it.  I will pickle the data for use in the next portion of the project.


In [36]:
import pickle

hoops_data.to_pickle('hoops_data.pickle')

I've also decided to add the change in each metric to individual records for later analysis

In [37]:
#use the stat diff function created earlier to add change in each stat to individuals (as opposed to teams)

bball_data['WORP_Chg'] = bball_data.apply(lambda row: get_stat_diff('WORP', bball_data, 'Player', 'Age', row), axis=1)
bball_data['VORP_Chg'] = bball_data.apply(lambda row: get_stat_diff('VORP', bball_data, 'Player', 'Age', row), axis=1)
bball_data['BPM_A_Chg'] = bball_data.apply(lambda row: get_stat_diff('BPM_A', bball_data, 'Player', 'Age', row), axis=1)
bball_data['PER_Chg'] = bball_data.apply(lambda row: get_stat_diff('PER', bball_data, 'Player', 'Age', row), axis=1)

In [38]:
bball_data = bball_data.dropna()
bball_data.head()

,Year,Tm,Player,Age,Use_Year,BPM,VORP,WORP,BPM_A,PER,G,WORP_Chg,VORP_Chg,BPM_A_Chg,PER_Chg
10,2011,ATL,Jeff Teague,22.0,2011,-3.9,-0.3,-1.066509,-0.712424,11.0,71.0,0.000000,0.0,0.000000,0.0
30,2012,ATL,Josh Smith,26.0,2012,2.9,3.3,9.327586,1.948806,19.2,77.0,-5.410306,-1.9,-1.844182,-1.8
31,2012,ATL,Jeff Teague,23.0,2012,-1.8,0.0,0.155495,-0.440396,14.6,70.0,1.222004,0.3,0.272027,3.6
32,2012,ATL,Joe Johnson,30.0,2012,0.0,1.3,3.187169,0.000000,16.4,72.0,-4.990367,-1.7,-1.613110,-2.9
33,2012,ATL,Zaza Pachulia,27.0,2012,-1.3,0.2,0.437940,-0.410874,12.4,79.0,-0.354585,-0.2,-0.217200,-0.7


In [39]:
print(bball_data[bball_data['Player'].str.contains('Horford')])

     Year   Tm      Player   Age  Use_Year  BPM  VORP       WORP     BPM_A  \
41   2012  ATL  Al Horford  25.0      2012  3.8   4.0  11.762890  2.610569   
54   2013  ATL  Al Horford  26.0      2013  2.5   0.5   1.398342  0.221037   
89   2014  ATL  Al Horford  27.0      2014  2.7   3.3   9.574574  1.890549   
108  2015  ATL  Al Horford  28.0      2015  2.3   1.0   3.101664  0.559807   

      PER     G   WORP_Chg  VORP_Chg  BPM_A_Chg  PER_Chg  
41   20.7  77.0   1.434812       0.4   0.369842      1.3  
54   19.0  11.0 -10.364548      -3.5  -2.389533     -1.7  
89   19.8  74.0   8.176232       2.8   1.669512      0.8  
108  22.0  29.0  -6.472910      -2.3  -1.330742      2.2  


In [40]:
# pickle the individual data for later use as well

bball_data.to_pickle('bball_data.pickle')